In [1]:
#### %pylab inline
# from scipy import interpolate, signal
# import time 
from obspy.core import read, Stream
from obspy.core.event import Catalog
import numpy as np
import matplotlib.pylab as plt
import os
import glob
from obspy.core.utcdatetime import UTCDateTime
# import urllib.request
from obspy import read_events, read_inventory
from obspy.signal import PPSD
# from obspy.io.jaxa_moon.core import _read_jaxa_moon
from statistics import mode
import io
import pandas as pd

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 4
plt.rcParams['lines.linewidth'] = 0.5
plt.rcParams['font.size'] = 12
SECONDS_PER_DAY=3600.*24


In [2]:
# read event catalog
# all of these take a while to run 


from obspy import read_events

xml_file = 'files/LunarCatalog_Nakamura_1981_and_updates_v1.xml'
print('Starting to read the catalog ...')
cat = read_events(xml_file)


print('done')
print(cat.count())



# cat.plot()


Starting to read the catalog ...
done
13058


In [3]:
print(cat.count())

13058


In [4]:
# make some catalogs for 1973 for each station

t1 = UTCDateTime('1973-01-01')
t2 = UTCDateTime('1973-12-31')
    
for station in ['S12', 'S14', 'S15', 'S16']:
    cat_all = Catalog()
    cat_deep = Catalog()
    cat_shallow = Catalog()
    cat_met = Catalog()
    cat_impact = Catalog()
    cat_unclassified = Catalog()
    
    for ev in cat: 
#         if ev.resource_id  not in ['smi:nakamura81/event/00011']:
#             continue
#         print(ev)
        
        picks = ev.picks
        
        picks_station = []
        amplitudes_station = []

        amplitudes = ev.amplitudes 

        for p in picks:   
            if p.waveform_id.station_code == station:

                if (p.time > t1 and p.time < t2):
                    picks_station.append(p)
                    for a in ev.amplitudes:
                        if a.pick_id == p.resource_id:
                            amplitudes_station.append(a)

    #                 amplitude.pick_id = pick.resource_id
#         print(len(picks_station))
        if len(picks_station) > 0:  
            ev_station = ev.copy()
            ev_station.picks = picks_station
            ev_station.amplitudes = amplitudes_station
            if ev_station.event_type == 'earthquake':    
                for desc in ev_station.event_descriptions:
                    if desc.text == 'deep moonquake':
                        cat_deep.append(ev_station)
                    elif desc.text == 'shallow moonquake':
                        cat_shallow.append(ev_station)
            if ev_station.event_type == 'meteorite':
                cat_met.append(ev_station)
            if ev_station.event_type == 'crash':
                cat_impact.append(ev_station)
                
            if ev_station.event_type == 'not reported':
                cat_unclassified.append(ev_station)
            cat_all.append(ev_station)
    
    cat_deep.write(filename='tmp_catalogs/1973_{}_deep.xml'.format(station), format='QUAKEML')            
#     cat_shallow.write(filename='catalogs/1973_{}_shallow.xml'.format(station), format='QUAKEML') 
    cat_met.write(filename='tmp_catalogs/1973_{}_met.xml'.format(station), format='QUAKEML') 
    if len(cat_impact) > 0: 
        cat_impact.write(filename='tmp_catalogs/1973_{}_impact.xml'.format(station), format='QUAKEML') 
    if len(cat_unclassified) > 0:
        cat_unclassified.write(filename='tmp_catalogs/1973_{}_unclassified.xml'.format(station), format='QUAKEML') 
        
print('finished')


finished


In [5]:
# make some catalogs for all years

for station in ['S12', 'S14', 'S15', 'S16']:

    print(station)
    cat_all = Catalog()
    cat_deep = Catalog()
    cat_shallow = Catalog()
    cat_met = Catalog()
    cat_impact = Catalog()
    
#     for ev in cat[7000:7001]:
    for ev in cat: 
#         if ev.resource_id  not in ['smi:nakamura81/event/10500']:
#             continue

        picks = ev.picks
        picks_station = []
        amplitudes_station = []

        amplitudes = ev.amplitudes 

        for p in picks:   
            if p.waveform_id.station_code == station:

#                 if (p.time > t1 and p.time < t2):
#                     print(p)
#                     break 
                    picks_station.append(p)
#                     print(ev.amplitudes)
                    for a in ev.amplitudes:
                        if a.pick_id == p.resource_id:
                            amplitudes_station.append(a)

        if len(picks_station) > 0:  
            ev_station = ev.copy()
            ev_station.picks = picks_station
            ev_station.amplitudes = amplitudes_station
            if ev_station.event_type == 'earthquake':    
                for desc in ev_station.event_descriptions:
                    if desc.text == 'deep moonquake':
                        cat_deep.append(ev_station)
                    elif desc.text == 'shallow moonquake':
                        cat_shallow.append(ev_station)
            if ev_station.event_type == 'meteorite':
                cat_met.append(ev_station)
            if ev_station.event_type == 'crash':
                cat_impact.append(ev_station)
            cat_all.append(ev_station)
           
    print(len(cat_shallow))
    cat_shallow.write(filename='tmp_catalogs/{}_shallow.xml'.format(station), format='QUAKEML') 
    
    print(len(cat_impact))
    cat_impact.write(filename='tmp_catalogs/{}_impact.xml'.format(station), format='QUAKEML') 
        
print('finished')



S12
24
9
S14
25
6
S15
25
4
S16
24
2
finished


In [6]:
# make catalogs for periods of operation in the flat mode 

# 1974-10-16T14:02:36.073–1975-04-09T15:31:03.702
# 1975-06-28T13:48:23.124–1977-03-27T15:41:06.247
for station in ['S12']:
    
    t1 = UTCDateTime('1974-10-16T14:02:36.073Z')
    t2 = UTCDateTime('1975-04-09T15:31:03.702')
    t3 = UTCDateTime('1975-06-28T13:48:23.124')
    t4 = UTCDateTime('1977-03-27T15:41:06.247')

    
    print(station)
    cat_all = Catalog()
    cat_deep = Catalog()
    cat_shallow = Catalog()
    cat_met = Catalog()
    cat_impact = Catalog()
    
#     for ev in cat[7000:7001]:
    for ev in cat: 
#         if ev.resource_id  not in ['smi:nakamura81/event/10500']:
#             continue

        

        picks = ev.picks
        picks_station = []
        amplitudes_station = []

        amplitudes = ev.amplitudes 

        for p in picks:   
            if p.waveform_id.station_code == station: 
                if (p.time > t1 and p.time < t2) or (p.time > t3 and p.time < t4):
                    picks_station.append(p)
#                     print(ev.amplitudes)
                    for a in ev.amplitudes:
                        if a.pick_id == p.resource_id:
                            amplitudes_station.append(a)

        if len(picks_station) > 0:  
            ev_station = ev.copy()
            ev_station.picks = picks_station
            ev_station.amplitudes = amplitudes_station
            if ev_station.event_type == 'earthquake':    
                for desc in ev_station.event_descriptions:
                    if desc.text == 'deep moonquake':
                        cat_deep.append(ev_station)
                    elif desc.text == 'shallow moonquake':
                        cat_shallow.append(ev_station)
            if ev_station.event_type == 'meteorite':
                cat_met.append(ev_station)
            if ev_station.event_type == 'crash':
                cat_impact.append(ev_station)
            cat_all.append(ev_station)
           

    if len(cat_shallow) > 0:
        cat_shallow.write(filename='tmp_catalogs/flat_{}_shallow.xml'.format(station), format='QUAKEML') 
        print('Shallow: ', len(cat_shallow))
    if len(cat_deep) > 0:  
        cat_deep.write(filename='tmp_catalogs/flat_{}_deep.xml'.format(station), format='QUAKEML')
        print('Deep: ', len(cat_deep))
    if len(cat_met) > 0:  
        cat_met.write(filename='tmp_catalogs/flat_{}_met.xml'.format(station), format='QUAKEML') 
        print('Meteoroid: ', len(cat_met))
    if len(cat_impact) > 0: 
        cat_impact.write(filename='tmp_catalogs/flat_{}_impact.xml'.format(station), format='QUAKEML') 
        print('Impact: ', len(cat_impact))
#     if len(cat_unclassified) > 0:
#         cat_unclassified.write(filename='tmp_catalogs/flat_{}_unclassified.xml'.format(station), format='QUAKEML') 
        
print('finished')


S12
Shallow:  8
Deep:  867
Meteoroid:  277
finished


In [7]:
print('End of Notebook')

End of Notebook
